In [ ]:
%load_ext autoreload
%autoreload 2

# View cellpy files

In [1]:
from pathlib import Path
import os
import pickle, importlib

import pandas as pd
from IPython.display import display

import cellpy

In [2]:
data_home = Path(r"..\..\cellpy_utilities\dev_data\cellpy_files")
assert data_home.is_dir()
all_files = os.listdir(data_home)

## Meta

In [3]:
def get_info(filename):
    meta_dir="/info"
    parent_level="CellpyData"
    with pd.HDFStore(filename) as store:
        print(" info ".center(83, "="))
        print(store.info())
        print(83*"-")
        parent_level = Path(store.keys()[0]).parent.name

        try:
            meta_table = store.select(parent_level + meta_dir)
        except KeyError:
            raise WrongFileVersion(
                "This file is VERY old - cannot read file version number"
            )
    print(" meta (first row) ".center(83, "="))
    print(meta_table.iloc[0, :].T)
    print(83*"-")
    meta_dict = meta_table.iloc[0, :].to_dict()
    return meta_dict

In [4]:
for f in all_files:
    print("\n")
    print(f)
    print("\n")
    get_info(data_home / f)



20160805_test001_45_cc.h5


======================================= info ======================================
<class 'pandas.io.pytables.HDFStore'>
File path: ..\..\cellpy_utilities\dev_data\cellpy_files\20160805_test001_45_cc.h5
/CellpyData/fid                frame        (shape->[1,10])                                                  
/CellpyData/info               frame        (shape->[1,34])                                                  
/CellpyData/raw                frame_table  (typ->appendable,nrows->10261,ncols->18,indexers->[index],dc->[])
/CellpyData/steps              frame_table  (typ->appendable,nrows->103,ncols->64,indexers->[index],dc->[])  
/CellpyData/summary            frame_table  (typ->appendable,nrows->18,ncols->31,indexers->[index],dc->[])   
-----------------------------------------------------------------------------------
================================= meta (first row) ================================
mass                                            

## Check why strange

In [5]:
tmp_dir = Path("tmp")
f_name = "20220106_marte_04_cc_01.h5"

In [6]:
f_original = data_home / f_name
f_new = tmp_dir / f_name

In [7]:
c = cellpy.get(f_original)

(cellpy) - the column header 'power_txt' not found
(cellpy) - the column header 'channel_id_txt' not found
(cellpy) - the column header 'data_flag_txt' not found
(cellpy) - the column header 'test_name_txt' not found
(cellpy) - missing key in meta_table: raw_unit_current
(cellpy) - missing key in meta_table: raw_unit_charge
(cellpy) - missing key in meta_table: raw_unit_voltage
(cellpy) - missing key in meta_table: raw_unit_time
(cellpy) - missing key in meta_table: raw_unit_resistance
(cellpy) - missing key in meta_table: raw_unit_power
(cellpy) - missing key in meta_table: raw_unit_energy
(cellpy) - missing key in meta_table: raw_unit_frequency
(cellpy) - missing key in meta_table: raw_unit_mass
(cellpy) - missing key in meta_table: raw_unit_nominal_capacity
(cellpy) - missing key in meta_table: raw_unit_specific_gravimetric
(cellpy) - missing key in meta_table: raw_unit_specific_areal
(cellpy) - missing key in meta_table: raw_unit_specific_volumetric
(cellpy) - missing key in meta_t

In [8]:
c.save(f_new)

In [9]:
c2 = cellpy.get(f_new)

In [10]:
get_info(f_new)

======================================= info ======================================
<class 'pandas.io.pytables.HDFStore'>
File path: tmp\20220106_marte_04_cc_01.h5
/CellpyData/fid                frame        (shape->[2,10])                                                  
/CellpyData/info               frame        (shape->[1,52])                                                  
/CellpyData/raw                frame_table  (typ->appendable,nrows->18998,ncols->26,indexers->[index],dc->[])
/CellpyData/steps              frame_table  (typ->appendable,nrows->226,ncols->64,indexers->[index],dc->[])  
/CellpyData/summary            frame_table  (typ->appendable,nrows->33,ncols->31,indexers->[index],dc->[])   
-----------------------------------------------------------------------------------
================================= meta (first row) ================================
mass                                                     1.2
channel_index                                           N

{'mass': 1.2,
 'channel_index': None,
 'channel_number': None,
 'creator': 'no_name',
 'cycle_mode': 'anode',
 'schedule_file_name': None,
 'start_datetime': Timestamp('2022-06-01 16:16:48'),
 'test_ID': None,
 'cell_no': None,
 'name': 'no_name',
 'nom_cap': 2000.0,
 'nom_cap_specifics': None,
 'material': 'silicon',
 'item_ID': None,
 'active_electrode_area': 1.0,
 'active_electrode_thickness': None,
 'electrolyte_type': None,
 'electrolyte_volume': None,
 'active_electrode_type': None,
 'counter_electrode_type': None,
 'reference_electrode_type': None,
 'experiment_type': None,
 'active_electrode_current_collector': None,
 'reference_electrode_current_collector': None,
 'comment': None,
 'cellpy_file_version': 7,
 'current_hard': 1e-13,
 'current_soft': 1e-05,
 'stable_current_hard': 2.0,
 'stable_current_soft': 4.0,
 'stable_voltage_hard': 2.0,
 'stable_voltage_soft': 4.0,
 'stable_charge_hard': 0.9,
 'stable_charge_soft': 5.0,
 'ir_change': 1e-05,
 'raw_unit_current': 'A',
 'raw_u

### Notes

Best way to load meta is to make it directly into a dictionary

## FID

In [11]:
a = "/cellpydata/fidtable"

In [12]:
"fid" in a

True

In [13]:
class PickleProtocol:
    """Context for using a specific pickle protocol."""

    def __init__(self, level):
        self.previous = pickle.HIGHEST_PROTOCOL
        self.level = level

    def __enter__(self):
        importlib.reload(pickle)
        pickle.HIGHEST_PROTOCOL = self.level

    def __exit__(self, *exc):
        importlib.reload(pickle)
        pickle.HIGHEST_PROTOCOL = self.previous


def pickle_protocol(level):
    return PickleProtocol(level)

In [21]:
PICKLE_PROTOCOL = 4
def get_fid(filename):
    print()
    print(filename)
    with pickle_protocol(PICKLE_PROTOCOL):
        with pd.HDFStore(filename) as store:
            try:
                print(store.keys())
                for n in store.keys():
                    if "fid" in n:
                        k = n
                        break
                else:
                    print("COUD NOT FIND IT")
                print(f"THE KEY: {k}")
                fid_table = store.select(k)
            except KeyError:
                raise WrongFileVersion(
                    "This file is VERY old - cannot read file version number"
                )
    display(fid_table)
    return fid_table

In [24]:
bad_files = [
    "20160805_test001_45_cc_01.h5",
    "20160805_test001_45_cc_v0.h5",
    "20160805_test001_45_cc_v4.h5"
]
for f in all_files:
    if f not in bad_files:
        try:
            get_fid(data_home / f)
        except Exception:
            print("NO")
# Note to self: looks like the whole cellpy code is inside some of the old hdf5 files
#   also, fid cannot be loaded from the old hdf5 files (0, 4) and the 20160805_test001_45_cc_01.h5 file
#   but the 20160805_test001_45_cc_01.h5 file is not used at all within cellpy


..\..\cellpy_utilities\dev_data\cellpy_files\20160805_test001_45_cc.h5
['/CellpyData/fid', '/CellpyData/info', '/CellpyData/raw', '/CellpyData/steps', '/CellpyData/summary']
THE KEY: /CellpyData/fid


,raw_data_name,raw_data_full_name,raw_data_size,raw_data_last_modified,raw_data_last_accessed,raw_data_last_info_changed,raw_data_location,raw_data_files_length,last_data_point,raw_data_fid
0,20160805_test001_45_cc_01.res,C:\scripts\cellpy\testdata\data\20160805_test0...,1634304,1.604935e+09,1.604935e+09,1.587121e+09,C:\scripts\cellpy\testdata\data,10261,10261,\n<fileID>\nfull name: C:\scripts\cellpy\testd...



..\..\cellpy_utilities\dev_data\cellpy_files\20160805_test001_45_cc_v5.h5
['/CellpyData/fid', '/CellpyData/info', '/CellpyData/raw', '/CellpyData/steps', '/CellpyData/summary']
THE KEY: /CellpyData/fid


,raw_data_name,raw_data_full_name,raw_data_size,raw_data_last_modified,raw_data_last_accessed,raw_data_last_info_changed,raw_data_location,raw_data_files_length,last_data_point,raw_data_fid
0,/Users/jepe/scripting/cellpy/testdata/data/201...,/Users/jepe/scripting/cellpy/testdata/data/201...,1613824,1.531764e+09,1.542483e+09,1.531764e+09,/Users/jepe/scripting/cellpy/testdata/data,10261,10261,\n<fileID>\nfull name: /Users/jepe/scripting/c...



..\..\cellpy_utilities\dev_data\cellpy_files\20160805_test001_45_cc_v6.h5
['/CellpyData/fid', '/CellpyData/info', '/CellpyData/raw', '/CellpyData/steps', '/CellpyData/summary']
THE KEY: /CellpyData/fid


...

,raw_data_name,raw_data_full_name,raw_data_size,raw_data_last_modified,raw_data_last_accessed,raw_data_last_info_changed,raw_data_location,raw_data_files_length,last_data_point,raw_data_fid
0,20160805_test001_45_cc_01.res,C:\scripts\cellpy\testdata\data\20160805_test0...,1634304,1.604935e+09,1.604935e+09,1.587121e+09,C:\scripts\cellpy\testdata\data,10261,10261,\n<fileID>\nfull name: C:\scripts\cellpy\testd...



..\..\cellpy_utilities\dev_data\cellpy_files\20220106_marte_03_cc_01.h5
['/CellpyData/fid', '/CellpyData/info', '/CellpyData/raw', '/CellpyData/steps', '/CellpyData/summary']
THE KEY: /CellpyData/fid


,raw_data_name,raw_data_full_name,raw_data_size,raw_data_last_modified,raw_data_last_accessed,raw_data_last_info_changed,raw_data_location,raw_data_files_length,last_data_point,raw_data_fid
0,20220106_marte_03_cc_01_Channel_30_Wb_1.xlsx,C:\scripting\cellpy\dev_data\raw\20220106_mart...,2623176,1.652363e+09,1.652466e+09,1.652001e+09,C:\scripting\cellpy\dev_data\raw,16760,16760,\n<fileID>\nfull name: C:\scripting\cellpy\dev...
1,20220106_marte_03_cc_01_Channel_30_Wb_2.xlsx,C:\scripting\cellpy\dev_data\raw\20220106_mart...,361853,1.652363e+09,1.652466e+09,1.652001e+09,C:\scripting\cellpy\dev_data\raw,2238,411710,\n<fileID>\nfull name: C:\scripting\cellpy\dev...



..\..\cellpy_utilities\dev_data\cellpy_files\20220106_marte_04_cc_01.h5
['/CellpyData/fid', '/CellpyData/info', '/CellpyData/raw', '/CellpyData/steps', '/CellpyData/summary']
THE KEY: /CellpyData/fid


,raw_data_name,raw_data_full_name,raw_data_size,raw_data_last_modified,raw_data_last_accessed,raw_data_last_info_changed,raw_data_location,raw_data_files_length,last_data_point,raw_data_fid
0,20220106_marte_04_cc_01_Channel_30_Wb_1.xlsx,C:\scripting\cellpy\dev_data\raw\20220106_mart...,2623178,1.652361e+09,1.652466e+09,1.652279e+09,C:\scripting\cellpy\dev_data\raw,16760,16760,\n<fileID>\nfull name: C:\scripting\cellpy\dev...
1,20220106_marte_04_cc_01_Channel_30_Wb_2.xlsx,C:\scripting\cellpy\dev_data\raw\20220106_mart...,361992,1.652363e+09,1.652466e+09,1.652279e+09,C:\scripting\cellpy\dev_data\raw,2238,411710,\n<fileID>\nfull name: C:\scripting\cellpy\dev...


# TODO

- remove 20160805_test001_45_cc_01.h5 from cellpy completely